In [2]:
import cairo
from datetime import datetime
import difflib
import extract_msg # conda install -c conda-forge msg-extractor
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd # conda install -c anaconda xlrd for read_excel
from pdf2image import convert_from_path # poppler-utils has to be installed along
from PIL import Image
import PyPDF2
import pytesseract # install tesseract & pytesseract both
import re
import string
import subprocess as sp
import tabula
import unicodedata

# MAIN

In [5]:
LAB_dict = {1: "RuferLab", 2: "SCITEC", 3: "Viteos", 4: "ABL Analytics", 5: "RWB analub", -1: "!!! UNKNOWN !!!"}

In [3]:
col_Analysis = ["ID", "ParamID", "EchantillonID", "Valeur", "AnalysisTime"]
col_Parameter = ["ID", "Nom", "Group", "Unit", "Limit"]
col_Echantillon = ["ID", "LocationID", "personID", "Code", "FloconsNb", "SamplingMethod", "Temperature"]
col_Location = ["ID", "Address", "WaterType", "Treatment", "Village", "Commune", "Distributeur"]#, "ParentLocID"] # for now do not take into account
col_Sampler = ["ID", "SamplerName"]

col_perAnalysis = col_Analysis[3:] + col_Parameter[1:]
col_perEchantillon = col_Echantillon[3:] + col_Location[1:-1] + col_Sampler[1:]
col_All = col_perAnalysis + col_perEchantillon

col_All

# Util

In [9]:
# Input: input_str : input string
# Output: final_str : input string without any accents
# https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    final_str = only_ascii.decode('utf-8')
    return final_str


# Input: fp : filepath
#        page_nb : page_nb-th page to be visualized
def visualize_pdf(fp, page_nb=-1):
    doc = convert_from_path(fp)
    if page_nb >= 0:
        display(doc[page_nb])
    else:
        for d in doc:
            display(d)
    
    return

# Input: fp : filepath
#        page_nb : text extracted from page_nb-th page with pytesseract
# Output: texts : list of text (per line)
def pytesseract_text(fp, page_nb=-1):
    doc = convert_from_path(fp)
    
    if page_nb >= 0:
        return list(pytesseract.image_to_string(doc[page_nb], lang='fra').split('\n'))
    else:
        texts = []
        for i in range(len(doc)):
            texts.append(list(pytesseract.image_to_string(doc[i], lang='fra').split('\n')))

        return texts
    
# Input: fp : filepath
# Output: pgNb : number of pages that the pdf contains
def get_nb_pages(fp):
    # https://stackoverflow.com/questions/49939085/xref-table-not-zero-indexed-id-numbers-for-objects-will-be-corrected-wont-con
    reader = PyPDF2.PdfFileReader(open(fp, mode='rb'), strict=False)
    pgNb = reader.getNumPages()
    
    return pgNb


# Input: fp : filepath
#        page_nb : -1 for 'all' else specified number of page on which the tables are extracted
# Output: tables: list of table extracted from the specified number of page (or the entire pdf)
def get_table(fp, page_nb=-1, multiple_tables=True, guess=True, output_format=None):
    # Extract 1 page
    if page_nb >= 0:
        tables = tabula.read_pdf(fp, pages=page_nb+1, stream=False, multiple_tables=multiple_tables, guess=guess, output_format=output_format)
        
    # Extract all the pages
    else:
        tables = tabula.read_pdf(fp, pages='all', stream=True, multiple_tables=multiple_tables, guess=guess, output_format=output_format)
    
    return tables

def extract_table(fp):
    pg_Nb = get_nb_pages(fp)
    dfs = get_table(fp, guess=True)
    
    # Case where one of the pages' tables are not correctly extracted
    if len(dfs) < pg_Nb:
        new_dfs = []
        for i in range(pg_Nb):
            tables = get_table(fp, page_nb=i, guess=True)
            if len(tables) == 0:
                tables = get_table(fp, page_nb=i, guess=False)
                if len(tables) == 0:
                    print("NO TABLES EXTRACTED FROM FILE : ", fp)
            new_dfs += tables
    
        if len(new_dfs) > len(dfs):
            return new_dfs

    return dfs


# Input: fp : filepath
#        page_nb : -1 for 'all' else specified number of page on which the tables are extracted
# Output: bboxes: list of bbox extracted from each detected table
def get_bbox_table(fp, page_nb=-1):
    # Only for getting the table bounding box purpose
    # https://blog.chezo.uno/a-recent-update-of-tabula-py-a923d2ab667b
    tables = get_table(fp, page_nb, output_format="json")
    bboxes = []
    for table in tables:
        top = table['top']
        left = table['left']
        bottom = table['height'] + top
        right = table['width'] + left
        bbox = [top, left, bottom, right]
        bboxes.append(bbox)
    
    return bboxes
    
# Add to a dictionary of type {k: [v]}
# k: key to which v has to be appended
# v: append to the list of the given k or create a new list [v] (if v is not already a list)
def add_item_to_dict_of_list(d, k, v):
    if k in d.keys():
        if isinstance(v, list):
            d[k] += v
        else:
            d[k].append(v)
    else:
        if isinstance(v, list):
            d[k] = v
        else:
            d[k] = [v]
        
    return d


def merge_two_columns_at_row(df, first_col_idx, second_col_idx, row_idx):
    first_val = df.at[row_idx, df.columns[first_col_idx]]
    second_val = df.at[row_idx, df.columns[second_col_idx]]
        
    if first_val is np.nan and second_val is np.nan:
        new_val = np.nan
        
    elif first_val is np.nan:
        new_val = second_val
    elif second_val is np.nan:
        new_val = first_val
    else:
        new_val = str(first_val) + " " + str(second_val)
    #if type(new_val) == str:
    #    df[df.columns[first_col_idx]] = df[df.columns[first_col_idx]].astype('str')
    df.at[row_idx, df.columns[first_col_idx]] = new_val
    
    return df


def merge_two_columns(df, first_col_idx, second_col_idx):
    for i in range(len(df)):
        df = merge_two_columns_at_row(df, first_col_idx, second_col_idx, i)
    
    df.drop(columns=df.columns[second_col_idx], axis=1, inplace=True)
    
    return df


def is_unnamed(c):
    return str(c).startswith("Unnamed")


def unify_NaN(df):
    nan_values = ['N/A', 'n/a']
    return df.replace(nan_values, np.nan)


# Use convert_from_path to check if it is a pdf file or not (ex) lab's logo img file)
def is_pdf(attachment):
    tmp = open("tmp.pdf", 'wb')
    tmp.write(attachment.data)
    tmp.close()
    
    out = True
    try:
        doc = convert_from_path("tmp.pdf")
    except Exception:
        out = False
        
    os.remove("tmp.pdf")
    
    return out


# MSG file extractor
# Note that sometimes, there exists msg files + respective pdf files -> Do not extract if pdf exists already
def extract_pdf_from_msg(filepath):
    print(filepath)
    
    # Skip wif the pdf file exists already
    if os.path.isfile(filepath[:-3]+"pdf"):
        return None

    filenames = []

    msg = extract_msg.Message(filepath)
    attachments = msg.attachments
    
    attachments = [att for att in attachments if (is_pdf(att))]
    if len(attachments) == 0:
        print("NO ATTACHMENT")
        return None
    
    for i, attachment in enumerate(attachments):
        filename = ".".join(filepath.split('.')[:-1])
        print(filename)
        if i>0 : 
            filename += "_"+str(i+1)
        file = open(filename+".pdf", 'wb')
        print("EXTRACT : ", filename)
        file.write(attachment.data)
        file.close()
        filenames.append(filename+".pdf")
        
    if len(filenames) > 1:
        print("ERROR : MORE THAN ONE PDF EXTRACTED FROM MSG FILE")
        print(filenames)
    
    return filenames[0]


# Check whether the given pdf is iamge-based or not searchable pdf
# https://stackoverflow.com/questions/55704218/how-to-check-if-pdf-is-scanned-image-or-contains-text
def check_image_based_pdf(fp):
    image_based = True
    
    output = sp.getoutput("ocrmypdf " + fp + " output.pdf")
    if not re.search("PriorOcrFoundError: page already has text!",output):
        image_based = True
        print("Uploaded scanned pdf")
    else:
        image_based = False
        print("Uploaded digital pdf")
    
    return image_based

def from_image_based_to_searchable_pdf(fp):
    !ocrmypdf $fp $fp --remove-background --force-ocr
    
    return fp


def header_to_row(df):
    return df.reset_index(drop=True).T.reset_index().T.reset_index(drop=True)

## Iterator

In [7]:
Autocontrole_PATH = "./Data/Autocontrole_EP/"

### 1. Sanitize the directory and file names

In [10]:
def sanitize_name_files(parent_dir):
    sub = [x for x in os.listdir(parent_dir) if not x.startswith(".")]
    
    if len(sub) == 0:
        return
    
    for c in sub:
        new_c = remove_accents(c.replace(" ", "_").replace(string.punctuation, "")).replace("'", "").replace("(", "").replace(")", "")
        #print(c)
        
        os.rename(parent_dir + c, parent_dir + new_c)
    
    return

### 2. create the pdf file for all the msg files & classify each document to its respective laboratory

In [11]:
def check_keyword(kw, txt):
    return np.any([True if kw in t.lower() else False for t in txt])

def get_laboratory(file):
    txt = pytesseract_text(file, page_nb=0)
    
    if check_keyword("ruferlab", txt):
        LAB = 1
    # To validate the keyword to check
    elif check_keyword("scitec", txt):
        LAB = 2
    elif check_keyword("viteos", txt):
        LAB = 3
    elif check_keyword("abl analytics", txt):
        LAB = 4
    elif check_keyword("rwb analub", txt):
        LAB = 5
    else:
        LAB = -1
        
    return LAB

In [10]:
#CODE to read a previously written text file containing the classified files list
RUFERLAB_FILES = []
SCITEC_FILES = []
VITEOS_FILES = []
ABL_RWB_FILES = []

FILES = [RUFERLAB_FILES, SCITEC_FILES, VITEOS_FILES, ABL_RWB_FILES]
fns = ["ruferlab_files.txt", "scitec_files.txt", "viteos_files.txt", "ablrwb_files.txt"]

for fn, ls in zip(fns, FILES):
    textfile = open(fn, "r")
    for l in textfile:
        ls.append(l.replace("\n", ""))
    textfile.close()

### 3. Data extraction per laboratory type

General methods applicable to both laboratories (ABL_RWB_LAB and RUFER)

In [12]:
def extract_sampler(text_ls, LAB):
    sampler_name = ""
    
    if LAB == 1 or LAB == 3:
        text_ls = [y for x in text_ls for y in x if y.strip() != '']
        line = [t for t in text_ls if "Nom du préleveur" in t or "préleveur" in t or "Nom du preleveur" in t]
        if len(line) > 0 :
            line = line[0]
            sampler_name = line.replace('Nom du préleveur', '').replace("Nom du preleveur", '').replace("préleveur", '').strip()
        
    if LAB == 4 or LAB == 5:
        # preprocess the text_ls
        text_ls = [x for x in text_ls if x.strip() != '']
        sampler_keyword = 'Prélevé '
        line = [t for t in text_ls if sampler_keyword in t][0].replace(sampler_keyword, "")
        if " le" in line:
            sampler_name = line[:line.index(" le")]
    
    if len(sampler_name.split()) < 2:
        return sampler_name
    
    sampler_name = sampler_name.split()
    firstname = sampler_name[0]
    lastname = sampler_name[1]

    #return lastname, firstname
    return lastname + " " + firstname


def extract_watertype(texts, LAB):
    watertype = ""
    
    watertype_keyword_dict = {1: "Nature de l'échantillon", 2: "", 3: "", 4: "Nature de l'échantillon: ", 5: "Nature de l'échantillon: "}
    
    watertype_keyword = watertype_keyword_dict[LAB]
    line = [t for t in texts if watertype_keyword in t]
    if len(line) == 0:
        print("extract_watertype(): Water Type Unfound")
        return None
    
    if LAB == 1:
        watertype = line[0].replace(watertype_keyword, '')
        
    if LAB == 4 or LAB == 5:
        watertype = line[0].replace(watertype_keyword, '')
    
    return watertype.strip()


def extract_date(texts, LAB=1):
    date = ""
    
    line = [t for t in texts if "" in t or "Date d'échantillonnage" in t]
    if len(line) == 0:
        print("extract_date(): Date Unfound")
        return None
    
    if "Date de prélèvement" in line[0]:
        date = line[0].replace("Date de prélèvement", '')
    elif "Date d'échantillonnage" in line[0]:
        date = line[0].replace("Date d'échantillonnage", '')
    
    return date.strip()
    

#### LAB : ABL_RWB_LAB

In [13]:
def filter_table_keyword(df, start="Analyses effectuees", end=["Commentaire:", "Toute reproduction"]):
    if start in df.values:
        start_idx = df[df == start].dropna(how='all').index[0]+1
        
        if end[0] in df.values:
            end_idx = df[df == end[0]].dropna(how='all').index[0]
            df = df[start_idx:end_idx].copy()
        # Check if Toute reproduction keyword is contained in one of the substring of df
        elif np.any([True for z in [y for x in df.values for y in x if not y is np.nan] if end[1] in str(z)]):
            
            end_idx = df.replace(np.nan, "").applymap(lambda x: end[1] in x).replace(False, np.nan).dropna(how='all').index[0]
            df = df[start_idx:end_idx].copy()
        else:
            df = df[start_idx:].copy()
            
        df.dropna(axis=1, how='all', inplace=True)
        df.columns = df.iloc[0].values
        df.drop(df.index[0], axis=0, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
    return df
    

def split_merged_columns(df, col, col_idx, col_name, keywords_dict):
    def get_common_key(s, ks, start=True):
        # replace any multiple whitespaces with single whitespace
        s = " ".join(s.split())
        for k in ks:
            if k in s:
                if start:
                    return k
                else:
                    return s.replace(k, "").strip()
            
        if start:
            return s.split()[0]
        else:
            return ' '.join(s.split()[1:])
    
    if col_name in keywords_dict.keys():
        keywords = keywords_dict[col_name]
        
        start_split = lambda x: get_common_key(x, keywords)
        end_split = lambda x: get_common_key(x, keywords, start=False)
    else:
        start_split = lambda x: x.split()[0]
        end_split = lambda x: ' '.join(x.split()[1:])
    # start split
    df.insert(col_idx, col_name, df[df.columns[col_idx]].map(start_split).values)
    # end_split
    df[col] = df[col].map(end_split)
    df = df.applymap(lambda x: np.nan if x == "" else x)
    df.rename({col: col.replace(col_name, "").strip()}, axis=1, inplace=True)
    
    return df


def format_ABL_RWB_df(df, cols):
    cols = ["Parametre", "Methode", "Date", "Resultat", "Unite"]
    col_keywords_dict = {"Parametre": ["Temperature mesuree in-situ", "Escherichia coli", "Enterocoques", "Germes aerobies", "Conductivite (25 C)", "Absorption UV 254"]}
    
    # Case where the result column has been split into two (because of < or <=)
    if len(df.columns) == len(cols)+1 and np.all([x in ["<", "<=", ">", ">=", "~"]for x in df[df.columns[3]].dropna().unique().tolist()]):
        df = merge_two_columns(df, 3, 4)
    
    # When we suppose the table is correctly extracted but without correct column names -> add the cols
    if len(df.columns) == len(cols) and not np.any([x in cols for x in df.columns.values]):
        # Swap eventual unnmaed column to NaN
        df.rename({c: np.nan for c in df.columns if is_unnamed(c)}, axis=1, inplace=True)
        # Shift the header to the row
        # https://stackoverflow.com/questions/53282075/turn-the-column-headers-into-the-first-row-and-row-headers-into-the-first-column
        df = df.reset_index(drop=True).T.reset_index().T
        df.columns = cols
        
        return df.reset_index(drop=True)
        
    # Create correct columns
    if len(df.columns) == len(cols) and np.equal(df.columns, cols).all():
        return df
    else:
        col_idx = 0
        for i, c in enumerate(cols):
            # Process NaN column : merge it
            # Then work with the given column name as this merge has not happened
            if df.columns[col_idx] is np.nan:
                df = merge_two_columns(df, col_idx-1, col_idx)

            if df.columns[col_idx] == c:
                col_idx += 1

            elif c in df.columns[col_idx]:
                df = split_merged_columns(df, df.columns[col_idx], col_idx, c, col_keywords_dict)
                if c == df.columns[col_idx].split()[-1]:
                    col_idx += 1
    
    return df


def extract_table_from_ABL_RWB_pdf(fp, debug=False, continuous_table=False):
    out_tables = []
    
    pg_Nb = get_nb_pages(fp)
    
    if debug:
        print(fp)
        print(pg_Nb)
        visualize_pdf(fp)

    dfs = get_table(fp, guess=True)
    
    # Case where one of the pages' tables are not correctly extracted
    if len(dfs) < pg_Nb:
        dfs = []
        for i in range(pg_Nb):
            tables = get_table(fp, page_nb=i, guess=True)
            if len(tables) == 0:
                tables = get_table(fp, page_nb=i, guess=False)
                if len(tables) == 0:
                    print("NO TABLES EXTRACTED FROM FILE : ", fp)
            dfs += tables

    for df in dfs:
        df = df.copy()
        df = df.applymap(lambda x: remove_accents(x) if isinstance(x, str) else x)
        
        ABL_RWB_Columns = ["Parametre", "Methode", "Date", "Resultat", "Unite"]
        
        df = filter_table_keyword(df)
        if debug:
            print("After Filter")
            display(df)
        df = format_ABL_RWB_df(df, cols=ABL_RWB_Columns)
        if debug:
            print("Final")
            display(df)
        out_tables.append(df)
        
    if np.all([np.equal(tab.columns, out_tables[0].columns).all() for tab in out_tables]):
        out = pd.concat(out_tables).reset_index(drop=True)
    
    return out


def extractor_ABL_RWB(filepath):
    LAB = 4 # As ABL_RWB
    
    table = extract_table_from_ABL_RWB_pdf(filepath)
    # Handle the case if the pdf is image based
    if len(table) == 0:
        if check_image_based_pdf(filepath):
            filepath = from_image_based_to_searchable_pdf(filepath)
            table = extract_table_from_ABL_RWB_pdf(filepath)

    text_ls = pytesseract_text(filepath)[0]
    texts = [x for x in text_ls if x.strip() != '']
    
    echantillon_code = extract_echantillon_code(texts, LAB)
    flocons_nb = extract_flocons_nb(texts, LAB)
    temperature = table[table.Parametre.str.contains("Température")].Resultat.values
    temperature = temperature[0] if len(temperature) > 0 else None
    address = extract_address(texts, LAB)
    watertype = extract_watertype(texts, LAB)
    treatment = extract_treatment(texts, LAB)
    sampler = extract_sampler(texts, LAB)
    
    print(echantillon_code, flocons_nb, temperature, address, watertype, treatment, sampler)
    
    table.drop(columns=["Methode"], inplace=True)
    table.rename(columns={"Resultat": "Value"}, inplace=True)
    
    table["Limit"] = None
    table["Code"] = echantillon_code
    table["FloconsNb"] = flocons_nb
    table["Temperature"] = temperature
    table["Address"] = address
    table["WaterType"] = watertype
    table["Treatment"] = treatment
    table["Sampler"] = sampler
    
    # For debugging purpose
    table["filepath"] = filepath
    
    return table


def extract_echantillon_code(texts, LAB=4):
    echantillon_code = ""
    
    echantillon_keyword = "échantillon n°"
    line = [t for t in texts if echantillon_keyword in t]
    if len(line) == 0:
        print("extract_echantillon(): Echantillon Unfound")
        return None
    
    if LAB == 1 or LAB == 4 or LAB == 5:
        line = line[0]
        echantillon_code = line[line.index(echantillon_keyword)+len(echantillon_keyword):].split()[0]
        
    return echantillon_code.strip()


def extract_flocons_nb(texts, LAB=4):
    flocons_nb = ""
    
    flocons_nb_keyword = "Nbre de flacons: "
    line = [t for t in texts if flocons_nb_keyword in t]
    if len(line) == 0:
        print("extract_flocons_nb(): Flocons Nb Unfound")
        return None
    
    if LAB == 4 or LAB == 5:
        flocons_nb = line[0].replace(flocons_nb_keyword, '')
    
    if flocons_nb.isdigit() and int(flocons_nb) > 0:
        flocons_nb = int(flocons_nb)
    
    return flocons_nb


# texts without treatment
def extract_address(texts, LAB=4):
    address = ""
    
    address_keyword = "Point de prélèvement: "
    if LAB == 1:
        address_keyword = "Point de prélèvement"
    line = [t for t in texts if address_keyword in t]
    if len(line) == 0:
        print("extract_address(): Address Unfound")
        return None
    next_line = texts[texts.index(line[0])+1]
    if LAB == 1 or LAB == 4 or LAB == 5:
        address = line[0].replace(address_keyword, '').strip()
        if len(next_line.strip()) > 0:
            address += ', ' + next_line
    
    return address.strip()


def extract_treatment(texts, LAB=4):
    treatment = ""
    
    treatment_keyword_dict = {1: "", 2: "", 3: "", 4: "Traitement utilisé: ", 5: "Traitement utilisé: "}
    
    treatment_keyword = treatment_keyword_dict[LAB]
    line = [t for t in texts if treatment_keyword in t]
    if len(line) == 0:
        print("extract_treatment(): Treatment Unfound")
        return None
    
    if LAB == 4 or LAB == 5:
        treatment = line[0].replace(treatment_keyword, '')
    
    return treatment.strip()

In [ ]:
buffer = []
done_ABL_RWB_fps = []
undone_ABL_RWB_fps = []
loop_idx = 0

for fp in ABL_RWB_FILES[loop_idx:]:
    print(fp)
    try:
        table = extractor_ABL_RWB(fp)
        buffer.append(table)
        done_ABL_RWB_fps.append(fp)
    except:
        undone_ABL_RWB_fps.append(fp)
    loop_idx += 1
    
df = pd.concat(buffer, ignore_index=True)
df.to_pickle("df_ABL_RWB.pkl")

textfile = open("undone_ABL_RWB.txt", "w")
for fp in undone_ABL_RWB_fps:
    textfile.write(fp + "\n")
textfile.close()

textfile = open("done_ABL_RWB.txt", "w")
for fp in done_ABL_RWB_fps:
    textfile.write(fp + "\n")
textfile.close()

#### LAB : RUFER LAB

In [14]:
def get_echantillon_dict(ocr_text):
    echantillons_dict = {}
    echantillon_w = "Point de prélèvement"
    
    # Check first page and retrieve all the echantillon till there is "Remarque"
    echantillon_w_start_idx = [True if echantillon_w in w else False for w in ocr_text[0]].index(True)
    echantillon_w_end_idx = [True if "Remarque" in w else False for w in ocr_text[0]].index(True)

    # Retrieve all echantillons by splitting it with :
    potential_echantillons = [w for w in ocr_text[0][echantillon_w_start_idx:echantillon_w_end_idx] if ":" in w]
    for echantillon in potential_echantillons:
        splitted = echantillon.split(":")
        code = splitted[-2].split()[-1]
        name = splitted[-1].strip()
        if code not in echantillons_dict.keys():
            echantillons_dict[code] = name
        else:
            newcode = str(int(list(echantillons_dict.keys())[-1])+1)
            echantillons_dict[newcode] = name
    
    return echantillons_dict


# Post-process table
# 0. Drop NaN column (with column header Unnamed) & Drop NaN row
# 1. Merge if a column has only 1 value with its preceding column
# 2. Merge if Unit column splitted
# 3. Result column splitted because of < or <=
def post_process_table(df, rufer_columns=None):
    df = df.copy()
    # Drop NaN columns
    # ! the column header has to be Unnamed too !
    for c in df.columns:
        if (is_unnamed(c) or c == ":") and len(df[c].dropna()) == 0:
            df.drop(columns=[c], inplace=True)
            
    # Drop NaN rows
    df.dropna(axis=0, how='all', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    
    # 1st case: one part of the preceding column has been expanded to a new columns
    # Typically such column has only one value in the column and all the other rows are NaN
    for j in range(3):
        for col_idx, c in enumerate(df.columns):
            unique_val = df[c].dropna()
            if len(unique_val) == 1:
                r_idx = unique_val.index[0]
                df = merge_two_columns(df, col_idx-1, col_idx)
                break
        
    # 2nd case: Unit columns separted into two distinct columns
    if rufer_columns:
        unite_col = rufer_columns.index("Unite")
    else:
        unite_col = len(df)-1
    
    Unite_keywords = ['C', '°C', 'ml', 'C /l', '°F', 'mg/l', 'ng/l', 'cm-1', 'UFC/100 ml']
    if len(df.columns) > unite_col+1 and np.any(df[df.columns[unite_col+1]].astype(str).map(lambda x: x in (Unite_keywords)).values):
    #if np.any([True if v.isin(Unite_keywords) else False for v in df[df.columns[unite_col+1]].values]):
        df = merge_two_columns(df, unite_col, unite_col+1)
        
    # 3rd case : the result column has been split into two (because of < or <=)
    last_idx = 0
    while(last_idx < len(df.columns)-1):
        tmp_idx = last_idx
        for col_idx, c in enumerate(list(df.columns)[last_idx:]):
            tmp_idx = col_idx
            tmp_vals = df[c].dropna().unique().tolist()
            if len(tmp_vals) > 0 and np.all([x in ["<", "<=", ">", ">=", "~"] for x in tmp_vals]):
                df = merge_two_columns(df, last_idx+col_idx, last_idx+col_idx+1)
                break
        last_idx += tmp_idx
    
    return df


def extract_information_rufer(df, ocr_text, echantillon_dict):
    df = df.copy()
    # Remove Heure de prelevement
    rm_idxs = list(df[df.Parametre.apply(lambda x: len(difflib.get_close_matches(str(x), ["Heure de prélevement"])) > 0)].index)
    if len(rm_idxs) > 0:
        df.drop(rm_idxs, inplace=True)
    
    # Extract FloconsNb
    nb_flacons = df[df.Parametre.apply(lambda x: len(difflib.get_close_matches(str(x), ["Nombre de flacons", "Nb flacons", "flacons"])) > 0)][["Code", "Value"]]
    nb_flacons_dict = dict(zip(nb_flacons["Code"], nb_flacons["Value"]))
    df["FloconsNb"] = df.Code.map(lambda x: nb_flacons_dict[x] if x in nb_flacons_dict.keys() else None)
    
    # Extract Temperature
    temperatures = df[df.Parametre.apply(lambda x: len(difflib.get_close_matches(str(x), ["Temperature"])) > 0)][["Code", "Value"]]
    temperatures_dict = dict(zip(temperatures["Code"], temperatures["Value"]))
    df["Temperature"] = df.Code.map(lambda x: temperatures_dict[x] if x in temperatures_dict.keys() else None)
    
    # Extract Treatment
    treatment = df[df.Parametre.apply(lambda x: len(difflib.get_close_matches(str(x), ["Traitement"])) > 0)][["Code", "Value"]]
    treatment_dict = dict(zip(treatment["Code"], treatment["Value"]))
    df["Treatment"] = df.Code.map(lambda x: treatment_dict[x] if x in treatment_dict.keys() else None)
    
    # Extract WaterType
    watertype = extract_watertype(ocr_text[0], LAB=1)
    df["WaterType"] = watertype
    
    if echantillon_dict:
        # Extract Address
        df["Address"] = df.Code.map(lambda x: echantillon_dict[x] if x in echantillon_dict.keys() else None)
    else:
        df["Address"] = extract_address(ocr_text[0], LAB=1)
    
    # Extract Sampler
    sampler = extract_sampler(ocr_text, LAB=1)
    df["Sampler"] = sampler
    
    # Extract Limit
    df["Limit"] = None
    
    return df


def extractor_Rufer(filepath, manual=None, typeII=False):
    Rufer_Columns_2021 = ["Parametre", "Methode", "Date", "Unite"]
    Rufer_Columns_TypeII = ["Parametre", "Methodd", "Value", "Unite"]
    
    dfs = extract_table(filepath)
    pgNb = get_nb_pages(filepath)

    #if check_image_based_pdf(filepath):
    #    new_filepath = from_image_based_to_searchable_pdf(filepath)
    #else:
    #    new_filepath = filepath
    new_filepath = filepath
    
    #doc = convert_from_path(new_filepath)
    ocr_text = pytesseract_text(new_filepath)
    
    dfs = get_table(new_filepath)
    
    echantillon_dict = None
    
    tmp = []
    if manual:
        tmp = manual
        echantillon_dict = get_echantillon_dict(ocr_text)
    # Case I : numerous echantillons per document
    elif pgNb > 1 and not typeII:
        echantillon_dict = get_echantillon_dict(ocr_text)
        
        for df in dfs:
            df = post_process_table(df, Rufer_Columns_2021)
            display(df)
            # Rename columns
            new_column_names = Rufer_Columns_2021 + list(echantillon_dict.keys())[:len(df.columns) - len(Rufer_Columns_2021)]
            df.columns = new_column_names + ['Value']
            df["Code"] = extract_echantillon_code(ocr_text, LAB=1)#'211946'
            '''df.columns = new_column_names
            present_ech = list(set(echantillon_dict.keys()).intersection(df.columns))
            for k in present_ech:
                df[k] = df[k].map(lambda x: (k, x))
            df["Value"] = df[present_ech].apply(tuple, axis=1)
            df = df.drop(columns=present_ech)
            df = df.explode("Value")
            df["Code"] = df["Value"].map(lambda x: x[0])
            df["Value"] = df["Value"].map(lambda x: x[1])'''
    
            display(df)
        
            tmp.append(df)

    # Case II : 1 echantillon per document
    else:
        # Assume two tables as one page
        if pgNb == 1:
            dfs[0].insert(1, "Dummy", [None for _ in range(len(dfs[0]))])
            for df in dfs:
                df = post_process_table(df)
                if len(df.columns) < len(Rufer_Columns_TypeII):
                    df.insert(len(df.columns), "Unite", [None for _ in range(len(df))])
                elif len(df.columns) > len(Rufer_Columns_TypeII):
                    df = merge_two_columns(df, len(Rufer_Columns_TypeII)-1, len(Rufer_Columns_TypeII))

                if len(df.columns) == len(Rufer_Columns_TypeII):
                    df.columns = Rufer_Columns_TypeII
                    df["Code"] = extract_echantillon_code(ocr_text, LAB=1)
                    tmp.append(df)
        else:
            dfs[0].insert(1, "Dummy", [None for _ in range(len(dfs[0]))])
            '''if len(dfs) > 2:
                for i in range(2, len(dfs)):
                    if len(dfs[i].columns) == 5:
                        #dfs[i].drop(list(dfs[i].columns)[2], axis=1, inplace=True)
                        display(dfs[i])'''
            
            for df in dfs:
                df = post_process_table(df)
                if len(df.columns) < len(Rufer_Columns_TypeII):
                    df.insert(len(df.columns), "Unite", [None for _ in range(len(df))])
                elif len(df.columns) > len(Rufer_Columns_TypeII):
                    df = merge_two_columns(df, len(Rufer_Columns_TypeII)-1, len(Rufer_Columns_TypeII))
                if len(df.columns) == len(Rufer_Columns_TypeII):
                    df.columns = Rufer_Columns_TypeII
                    df["Code"] = extract_echantillon_code(ocr_text[0], LAB=1)
                    tmp.append(df)
        
            display(df)

    df = pd.concat(tmp).reset_index(drop=True)
    
    date = extract_date(ocr_text[0], LAB=1)
    df["Date"] = date
    df["Limit"] = None
    
    df = extract_information_rufer(df, ocr_text, echantillon_dict)
    if "Methode" in df.columns:
        df.drop(columns=["Methode"], inplace=True)
    
    # For debugging purpose
    df["filepath"] = filepath
    
    # Date NA fill
    df.Date = df.Date.fillna(method="ffill").fillna(method="bfill")
    
    return df[["Parametre", "Date", "Value", "Limit", "Unite", "Code", "FloconsNb", "Temperature", "Address", "WaterType", "Treatment", "Sampler", "filepath"]]

In [ ]:
buffer = []
done_Rufer_fps = []
undone_Rufer_fps = []
loop_idx = 0

for fp in RUFERLAB_FILES[loop_idx:]:
    print(fp)
    try:
        table = extractor_Rufer(fp, manual=tmps, typeII=False)
        buffer.append(table)
        done_Rufer_fps.append(fp)
        loop_idx += 1
    except Exception as e:
        print(e)
        undone_Rufer_fps.append(fp)

df = pd.concat(buffer, ignore_index=True)
df.to_pickle("df_Rufer.pkl")

textfile = open("undone_Rufer.txt", "w")
for fp in undone_Rufer_fps:
    textfile.write(fp + "\n")
textfile.close()

textfile = open("done_Rufer.txt", "w")
for fp in done_Rufer_fps:
    textfile.write(fp + "\n")
textfile.close()

## Data Post-processing

In [147]:
df_rufer = pd.read_pickle("df_Rufer.pkl")
df_abl = pd.read_pickle("df_ABL_RWB.pkl")

df = pd.concat([df_abl, df_rufer], ignore_index=True)
df

,Parametre,Date,Value,Unite,Limit,Code,FloconsNb,Temperature,Address,WaterType,Treatment,Sampler,filepath,Village,Commune,Distributeur
0,Température mesurée in-situ,10.04.12,11.7,°C,None,1892,1,11.7,"Bassecourt, eau de réseau, Espace Industriel 3...",Eau brute,UV,None,./Data/Autocontrole_EP/Bassecourt/2012/01892.pdf,NaN,NaN,NaN
1,Escherichia coli,11.04.12,0,germes/100ml,None,1892,1,11.7,"Bassecourt, eau de réseau, Espace Industriel 3...",Eau brute,UV,None,./Data/Autocontrole_EP/Bassecourt/2012/01892.pdf,NaN,NaN,NaN
2,Enterocoques,11.04.12,0,germes/100ml,None,1892,1,11.7,"Bassecourt, eau de réseau, Espace Industriel 3...",Eau brute,UV,None,./Data/Autocontrole_EP/Bassecourt/2012/01892.pdf,NaN,NaN,NaN
3,Germes aerobies,11.04.12,0,germes/ml,None,1892,1,11.7,"Bassecourt, eau de réseau, Espace Industriel 3...",Eau brute,UV,None,./Data/Autocontrole_EP/Bassecourt/2012/01892.pdf,NaN,NaN,NaN
4,Température mesurée in-situ,10.04.12,12.7,°C,None,1893,1,12.7,"Bassecourt, eau de réseau, Rue abbé Monnin, FMB",Eau de nappe brute,UV,None,./Data/Autocontrole_EP/Bassecourt/2012/01893.pdf,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21160,Chlore libre,,NaN,0.02,None,180778,NaN,NaN,"Réservoir 1500 m° Z1, Remarque",Eau,NaN,Stalder Joël,./Data/Autocontrole_EP/Boncourt/2018/20180427/...,NaN,NaN,NaN
21161,Conditions météo,,NaN,Beau,None,180778,NaN,NaN,"Réservoir 1500 m° Z1, Remarque",Eau,NaN,Stalder Joël,./Data/Autocontrole_EP/Boncourt/2018/20180427/...,NaN,NaN,NaN
21162,Escherichia Coli,,0,germes/100 ml,None,180778,NaN,NaN,"Réservoir 1500 m° Z1, Remarque",Eau,NaN,Stalder Joël,./Data/Autocontrole_EP/Boncourt/2018/20180427/...,NaN,NaN,NaN
21163,Entérocoques,,NaN,germes/100 ml,None,180778,NaN,NaN,"Réservoir 1500 m° Z1, Remarque",Eau,NaN,Stalder Joël,./Data/Autocontrole_EP/Boncourt/2018/20180427/...,NaN,NaN,NaN


#### Add Village, Commune, Distributeur

In [153]:
listing = pd.read_excel("Data/List_commune-distributeurs_EP.xlsx")
listing.dropna(how='all', axis=1, inplace=True)
listing.dropna(how='all', axis=0, inplace=True)
listing.fillna(method="ffill", inplace=True)
listing[["Commune", "Localite"]] = listing[["Commune", "Localite"]].applymap(lambda x: remove_accents(x.replace(" ", "_").replace(string.punctuation, "")).replace("'", "").replace("(", "").replace(")", "").rstrip())


def get_commune(path, listing):
    dir_name = path.split("/")[3]
    commune = dir_name
    village = None
    distributeur = None
    
    if dir_name in listing.Commune.values:
        locs = listing[listing.Commune == dir_name].Localite.values
        if dir_name in locs:
            village = dir_name
        elif len(locs) == 1:
            village = locs[0]
        else:
            pass
    elif dir_name in listing.Localite.values:
        village = dir_name
        commune = list(listing[listing.Localite == dir_name].Commune.values)[0]
    
    distributeur = list(listing[listing.Commune == commune].Distributeur.values)[0] if len(listing[listing.Commune == commune]) > 0 else commune
    
    return village, commune, distributeur


df["Village"] = df.filepath.map(lambda x: get_commune(x, listing))
df["Commune"] = df["Village"].map(lambda x: x[1])
df["Distributeur"] = df["Village"].map(lambda x: x[2])
df["Village"] = df["Village"].map(lambda x: x[0])

In [149]:
df.Address = df.Address.map(lambda x: x.split("Remarque")[0].strip() if x else None)
code_Address = df.loc[df.Address[df["Address"].str.contains(":").replace({None: False})].index]
for i, row in code_Address.iterrows():
    if not row.Code:
        df.at[i, "Code"] = row.Address.split(":")[0].strip()
df.Address[df["Address"].str.contains(":").replace({None: False})] = df.Address[df["Address"].str.contains(":").replace({None: False})].map(lambda x: x.split(":")[1].strip())
df.Address = df["Address"].map(lambda x: x.split(", (identification")[0] if x else x).map(lambda x: x.split(", identification")[0] if x else x)


In [150]:
def water_type_from_address(address):
    water_type = None
    if "Source" in address or "Fontaine" in address:
        water_type = "Eau brute"
    
    elif "traitement" in address:
        water_type = "Eau traitee"
    
    elif "Réservoir" in address or "Réservair" in address:
        water_type = "Reservoir"
    
    elif "Réseau" in address or "réseau" in address or "Réeau" in address:
        water_type = "Reseau"
        
    return water_type

df["WaterType"] = df["WaterType"].replace({'Eau': None, "Eeu": None, "eau": None, 'Eau de source brute': 'Eau brute', 'Eau de nappe brute': 'Eau brute', 'Eau de nappe filtrée': 'Eau brute', 'Réseau de Porrentruy': "Reseau", 'Eau de réseau Porrentruy': "Reseau", "Eau de boisson": "Reseau", "Eau potable": "Reseau", "Eau de source": "Eau brute", "Eau traitée": "Eau traitee"})
df["WaterType"] = df["WaterType"].replace({'eau de réseau': 'Reseau', 'Eau de source désinfectée': "Eau traitee", 'eau SEHA': 'Reseau', 'Eau SEV': 'Reseau', 'Eau SIVAMO': 'Reseau', 'Eau 2908 Grandfontaine': None})
for i, row in df.iterrows():
    if row.Address and not row.WaterType:
        df.at[i, "WaterType"] = water_type_from_address(row.Address)
    
    if row.WaterType and "filtrée" in str(row.WaterType) or "désinfectée" in str(row.WaterType) or "désinfetée" in str(row.WaterType):
        df.at[i, "WaterType"] = "Eau traitee"
    elif row.WaterType and "nappe" in str(row.WaterType) or "brute" in str(row.WaterType):
        df.at[i, "WaterType"] = "Eau brute"
    elif row.WaterType and ("réseau" in str(row.WaterType) or "boisson" in str(row.WaterType)):
        df.at[i, "WaterType"] = "Reseau"
        
df["WaterType"].unique()

array(['Eau brute', 'Eau traitee', None, 'Reseau', 'Reservoir'],
      dtype=object)

## Data Cleaning

In [151]:
def correct_input(df, col, false_kw, corrected_kw):
    idxs = df[col][df[col] == false_kw].index
    for i in idxs:
        df.at[i, col] = corrected_kw
    return df

In [ ]:
df_Parameter_corr = pd.read_excel("parameter_corrected.xlsx")
df_Parameter_corr.drop(columns="ID", inplace=True)
df_Parameter_corr = df_Parameter_corr.drop(df_Parameter_corr[df_Parameter_corr.Remarque == "à supprimer"].index).drop(columns="Remarque").reset_index(drop=True)
df_Parameter_corr = df_Parameter_corr.assign(Unit=df_Parameter_corr["Correct Unit"].fillna(df_Parameter_corr["Unit"]))
df_Parameter_corr = df_Parameter_corr.drop(columns="Correct Unit")
df_Parameter_corr.Nom = df_Parameter_corr.Nom.map(lambda x: remove_accents(x))
df_Parameter_corr.drop_duplicates(inplace=True)
df_Parameter_corr.at[df_Parameter_corr[df_Parameter_corr["Nom"] == "Conductivite (25 C)"].index, "Nom"] = "Conductivite (25°C)"
df_Parameter_corr.at[df_Parameter_corr[df_Parameter_corr["Nom"] == "Conductivite in situ 20C"].index, "Nom"] = "Conductivite in situ 20°C"
for i, row in df_Parameter_corr.iterrows():
    if row.Nom.startswith("-"):
        df_Parameter_corr.at[i, "Nom"] = row.Nom[1:].strip()
df_Parameter_corr.drop_duplicates(inplace=True)
df_Parameter_corr.drop([26, 27, 96, 97, 98, 99, 100, 101], inplace=True)
df_Parameter_corr.reset_index(drop=True, inplace=True)
df_Parameter_corr.drop([26, 40, 144], inplace=True)
df_Parameter_corr.reset_index(drop=True, inplace=True)
df_Parameter_corr.to_pickle("df_Parameter_corr.pkl")
for i in range(0, len(df_Parameter_corr), 50):
    display(df_Parameter_corr.loc[i:i+50])

In [ ]:
tmp = df.Parametre.map(lambda x: difflib.get_close_matches(str(x), list(df_Parameter_corr.Nom.unique())))

Param_to_clean = df.loc[tmp[tmp.str.len()==0].index]
Param_to_clean.Value = Param_to_clean.Value.replace({np.nan: None, None: None, "NaN": None, "n/a": None})
Param_to_clean = Param_to_clean[Param_to_clean.Value.notna()]
Param_to_clean = Param_to_clean[Param_to_clean.Value != 999]

to_drop_idx = []
for i, row in Param_to_clean.iterrows():
    new_Param_name = row.Parametre
    if row.Parametre == 'Carbone organique dissous' or row.Parametre == 'Carbone organique dissous (DOC)':
        df.at[i, "Parametre"] = "DOC"
    elif row.Parametre.startswith("Escherichia"):
        df.at[i, "Parametre"] = "Escherichia"
    elif len(row.Parametre.split())>1 and (row.Parametre.split()[1].startswith("NF") or row.Parametre.split()[1].startswith("EPA") or row.Parametre.split()[1].startswith("7.2-MOD")):
        df.at[i, "Parametre"] = row.Parametre.split()[0]
    
    new_name = difflib.get_close_matches(str(new_Param_name), list(df_Parameter_corr.Nom.unique()))
    if len(new_name) == 0:
        to_drop_idx.append(i)

In [ ]:
df.Parametre = df.Parametre.map(lambda x: difflib.get_close_matches(str(x), list(df_Parameter_corr.Nom.unique())))
df.drop(tmp[tmp.str.len()==0].index, inplace=True)
df.Parametre = df.Parametre.map(lambda x: x[0])
df.reset_index(drop=True, inplace=True)

In [ ]:
df.Limit = df.Parametre.map(lambda x: dict(zip(df_Parameter_corr.Nom, df_Parameter_corr.Limit))[x])
df.Unite = df.Parametre.map(lambda x: dict(zip(df_Parameter_corr.Nom, df_Parameter_corr.Unit))[x])

In [298]:
df = correct_input(df, "Date", 'Mercredi 78.08.2019', 'Mercredi 07.08.2019')
df = correct_input(df, "Date", '00:00:00', "01.01.99")
df = correct_input(df, "Date", 'RP iPiPiPiPSaimPedlii P3i1.P0i8.P2i01P9 iPirPeiPirmPipPirPirPeirmirir', 'Vendredi 30.08.2019')
df.Date = df.Date.replace({"": None})
df.Date.fillna("01.01.99", inplace=True)
df.Date = df.Date.map(lambda x: "01.01.99" if type(x)!= str else x)
df.Date = df.Date.map(lambda x: str(x).split()[-1]).map(lambda x: datetime.strptime(x, "%d.%m.%y") if x != "nan" and len(x) ==8 else x).map(lambda x: datetime.strptime(x, "%d.%m.%Y") if x != "nan" and type(x)==str else x)


In [312]:
for i, row in df.iterrows():
    if str(row.Value).startswith("<") or str(row.Value).startswith(">"):
        newval = row.Value.split()[-1]
        newval = newval.replace("<", "").strip()
        newval = newval.replace(">", "").strip()
        df.at[i, "Value"] = newval

In [313]:
df.Value = df.Value.map(lambda x: x.split()[-1] if x and type(x)==str else x)
df.Value = df.Value.map(lambda x: float(x) if type(x) == str and x.isnumeric() else x)
df.Value = df.Value.map(lambda x: 999 if type(x) == str else x)
df.Value = df.Value.fillna(999) #.Value.map(lambda x: 999 if type(x) == str else x)


In [314]:
df.Limit = df.Limit.fillna(999) # fill with impossible value to fit the dtype int

In [316]:
df.Temperature = df.Temperature.replace({'n/a': None, '10°C': None, np.nan: None, 'N/A': None, 'n/a nan': None, 'n/a °C': None, '&': None, '°c': None, '/ °C': None, 'Eau de javelE  au 10': 10, '-3°C': 0, '9?': 9, '°C': None})
df.Temperature = df.Temperature.replace("n/a", 999)
df.Temperature = df.Temperature.map(lambda x: x.split()[0] if x and type(x)!=float and type(x)!=int else x)#.map(lambda x: x/10.0 if x and int(x) > 50 else x)
df.Temperature = df.Temperature.map(lambda x: float(str(x).replace(',', '.')) if x else x)
df.Temperature = df.Temperature.map(lambda x: x/10.0 if x and x>40 and x!= 999 else x)
df.Temperature = df.Temperature.astype(float)
df.Temperature = df.Temperature.replace("n/a", 999).fillna(999).astype(float)
df.Temperature.unique()

array([ 11.7 ,  12.7 , 999.  ,  14.  ,  11.  ,  12.  ,  15.  ,  25.  ,
        17.  ,  21.  ,   7.  ,  17.1 ,  18.3 ,   9.6 ,   8.7 ,  12.3 ,
         9.3 ,   9.4 ,  12.5 ,  14.5 ,  13.5 ,  16.5 ,   8.5 ,  16.  ,
        18.  ,  18.5 ,  19.  ,  11.5 ,  10.  ,   9.5 ,  10.5 ,   9.  ,
        15.5 ,  10.2 ,  13.  ,  17.5 ,  15.2 ,  20.  ,   8.2 ,  15.8 ,
        15.3 ,   9.2 ,  15.7 ,  13.3 ,  13.2 ,   9.9 ,  12.2 ,  15.1 ,
        20.5 ,  16.4 ,  13.4 ,   7.2 ,  13.9 ,  12.1 ,   9.7 ,  16.2 ,
        15.4 ,  14.1 ,   7.7 ,   8.  ,   6.  ,   5.3 ,  12.4 ,   6.8 ,
        18.8 ,  10.6 ,   4.3 ,  14.8 ,  18.6 ,  20.4 ,  17.4 ,  19.8 ,
        13.1 ,  10.1 ,  13.6 ,  12.8 ,  18.2 ,  14.6 ,   8.8 ,   7.1 ,
        11.4 ,  11.1 ,  17.9 ,   7.4 ,   8.3 ,  17.2 ,  13.8 ,  12.6 ,
         9.8 ,  10.9 ,  10.7 ,  17.7 ,  10.8 ,   6.7 ,   5.  ,   6.5 ,
         4.  ,   7.5 ,  19.5 ,   8.1 ,   7.9 ,   7.25,   7.3 ,   7.8 ,
        10.3 ,   9.1 ,  15.9 ,  11.6 ,  18.9 ,  21.5 ,  20.2 ,  22.6 ,
      

In [317]:
df.Code.fillna(999, inplace=True)
df.Code = df.Code.map(lambda x : x.replace('?', '' ).replace('U', '') if x and type(x)==str else x)
df.Code = df.Code.astype(int)

In [318]:
df.FloconsNb = df.FloconsNb.replace({'1 1 1': None, np.nan: None, '1 1': None, '1 nan': None, '1 1 2 1': None, ')': None, '_': None, '1.04 201': 1, 'l': 1})
df.FloconsNb.fillna(999, inplace=True)
df.FloconsNb = df.FloconsNb.astype(int)

In [319]:
df.Treatment.unique()

array(['UV', 'dioxyde de chlore + UF', None, 'dioxyde de chlore',
       'ozone, eau de javel', 'UF', 'ozone', 'UV, dioxyde de chlore',
       'UV, eau de javel', 'eau de javel', 'dioxyde de chlore, eau de',
       'UV, dioxyde de chlore,UF', 'eau de javel, UF',
       'ultra-filtration+eau de javel', 'ultra-filtrationteau de javel',
       'UV, ozone', 'eau de javel, UV', 'ozone, dioxyde de chlore',
       'charbon actif', 'UF, UV', 'dioxyde de Chlore, UF',
       'dioxyde de chlore, chlore', 'eau de secours Porrentruy', 'None',
       'UV et UF', 'brute', 'Javel + UF', 'Chlore/UF', 'onore/ UV UV',
       'cores', 'Javel/UF', 'Chlore et UF', '+ chlore', 'Chlore + UV',
       '15.2 UV', '13.1 UV', '18.5 UV', '20.4 UV', '10.0 n/a',
       'Ozone /javel', 'Ozone /Javel', 'Brute', 'Chlore', 'Ozone / Javel',
       'Ozone et javel', 'UV/UF', 'Chlore /UF', 'uv', 'nia', '8 UV',
       'UF/UV', 'UV / UF', 'Ozone', 'Ozone / Chlore', 'Javel',
       'Ozone/eau de javel', 'n/a nan', 'Eau brute',

In [320]:
df.Treatment = df.Treatment.replace({np.nan: None, '8.0 UV': "UV", 'n/a': None, '16.0 16.0': None, '16.5 UV' : 'UV', '10 Brute': 'Brute', '9.3 Ozone/UF': 'Ozone/UF', 'Jave/ lUV': 'Javel/UV', "15 javel javell 14 brute 12.5 UF 16": 'Javel', 'Chlore Chlore total libre 0.00 0.04 mg/lmg/l': 'Chlore', '|': None, 'nuageux': None, 'n/a UF': 'UF'})
df.Treatment.fillna("None", inplace=True)
df.Treatment = df.Treatment.map(lambda x: re.sub(r'[0-9]', '', x) if type(x)==str else x)
df.Treatment = df.Treatment.map(lambda x: x.replace('n/a', '') if type(x)==str else x)
df.Treatment = df.Treatment.map(lambda x: x.replace('+', '/') if type(x)==str else x)
df.Treatment = df.Treatment.map(lambda x: x.replace('et', '/') if type(x)==str else x)
df.Treatment = df.Treatment.map(lambda x: x.replace('.', '') if type(x)==str else x)
df.Treatment = df.Treatment.map(lambda x: x.replace(' / ', '/').replace('/ ', '/').replace(' /', '/')).map(lambda x: x.replace('.', ''))
df.Treatment = df.Treatment.replace({'. mg/L': 'None', 'uv': 'UV', 'Eau brute': 'None', '': 'None'})
df.Treatment = df.Treatment.map(lambda x: x.strip())

df.Treatment.fillna("None", inplace=True)

In [330]:
df.to_pickle("df.pkl")

In [332]:
df.Sampler = df.Sampler.map(lambda x: remove_accents(x) if x else x)

In [390]:
df.Sampler.unique()

array([None, 'Jacques Heyer', 'Yves Salomon', 'Pierre Stieger',
       'Allemann J.-P.', 'Luginbuhl Didier', 'Pierre-Andre Guerdat',
       'Michel Chevre', 'Michel Dupre', 'Chapuis Serge', 'Serge Gschwind',
       'Raymond Voillat', 'Joel Stalder', 'Jean-Pierre Pataoner',
       'Francis Droz', 'Michel Wermeille', 'Fernando Ribeiro',
       'Florent Schori', 'Andre Chappuis', 'Valli Ugo', 'Goudron Vincent',
       'Falbriard Gabriel', 'F. G.', 'Jean-Luc Domine', 'Herve Gerster',
       'Gregory Jeannerat', 'Marchand Jacques', 'Allemann Blaise',
       'Fluck K.', 'Hauser Stephane', 'Panier Didier', 'Pierre Petignat',
       'Didier Panier', 'Christian Schneider', 'Francois Hernandez',
       'Corbat, C.', 'Corbat C.', 'Andre Vauclair', 'Ernest Monin',
       'Damien Belet', 'Dominique Queloz', 'Alain Baume',
       'Jean-Pierre Coulon', 'Gerard Coulon', 'Clerc Paul',
       'Mathieu Grossenbacher', 'Roth M.', 'Paul Crelin', 'Paul Crelier',
       'Serge Chapuis', 'Johann Stalder', 'Ch

In [391]:
df.Sampler = df.Sampler.replace({'Belet Damien': 'Damien Belet', 
                                 'Belet D.': 'Damien Belet',
                                 'Chappuis Andre': 'Andre Chappuis',
                                 'S.Chapuis': 'Andre Chappuis',
                                 'Chapuis M.': 'Andre Chappuis',
                                 'M. Chevre' : 'Michel Chevre',
                                '/ Chevre' : 'Michel Chevre',
                                'MChevre' : 'Michel Chevre',
                                 'Chevre M.': 'Michel Chevre', 
                                'Chevre Richard' : "Richard Chevre",
                                 'Chevre R.': "Richard Chevre",
                                 'Coulon JP': 'Jean-Pierre Coulon', 
                                 'Coulon Jean-Pierre': 'Jean-Pierre Coulon',
                                 'Pierre Jean': 'Jean-Pierre Coulon', 
                                 'Jean-Pierre Monsieur': 'Jean-Pierre Coulon', 
                                 'G.Coulon': 'Jean-Pierre Coulon',
                                 'Coulon Jean-Piere': 'Jean-Pierre Coulon', 
                                 'Couion J.-P.': 'Jean-Pierre Coulon', 
                                 'Coulon J.-P.': 'Jean-Pierre Coulon',
                                 'Domine J.-L.': 'Jean-Luc Domine', 
                                 'J-Luc Domine': 'Jean-Luc Domine', 
                                 'Domine Jean-Luc': 'Jean-Luc Domine', 
                                 'DOMINE Jean-Luc': 'Jean-Luc Domine',
                                 'Jean-Luc Dornine': 'Jean-Luc Domine', 
                                 'Jean-Luc Doraine': 'Jean-Luc Domine',
                                 'Dornine M.': 'Jean-Luc Domine',
                                 'Droz F.': 'Francis Droz',
                                 'Droz Francis': 'Francis Droz',
                                 'F.Droz/F.Schori': 'Francis Droz',
                                 'F.Schori F.Droz,': 'Francis Droz',
                                 'et F.Drez': 'Francis Droz',
                                 'F.Schori/F.Droz': 'Francis Droz',
                                 'Francis/Schori Droz': 'Francis Droz',
                                 'Fulgano Febio': 'Fabio Fulgano',
                                 'Fulgano Febio': "Fabio Fulgano",
                                 'M.Grossenbacher' : 'Marc Grossenbacher',
                                'Grossenbacher Marc' : 'Marc Grossenbacher',
                                'Grossenbacher M.' : 'Marc Grossenbacher',
                                'Grossenbacher M' : 'Marc Grossenbacher',
                                'Pierre Guerdat' : 'Pierre-Andre Guerdat',
                                 'J. Heyer': 'Jacques Heyer', 
                                 'Jacquet Heyer': 'Jacques Heyer',
                                 'J Heyer' : 'Jacques Heyer',
                                 'FHernandez': 'Francois Hernandez',
                                 'François Hernandez': 'Francois Hernandez',
                                 'F.Hernandez': 'Francois Hernandez',
                                 'Hernandez F.': 'Francois Hernandez',
                                 'F. Hernandez': 'Francois Hernandez',
                                 'Hernandez Francois': "Francois Hernandez",
                                 'Jeannerat Gregory': 'Gregory Jeannerat', 
                                 'G.Jeannerat': 'Gregory Jeannerat', 
                                 'Jeannerat G.': 'Gregory Jeannerat',
                                 'Joray A.': 'Andre Joray',
                                 'AA.Joray': 'Andre Joray',
                                 'Joray A,': 'Andre Joray',
                                 'A.Joray' : 'Andre Joray',
                                 'Joray Andre' : 'Andre Joray',
                                 'Marchand J.': 'Marchand Jacques',
                                 'Klay Stephane': 'Stephane Klay',
                                 'S.K!ay': 'Stephane Klay', 
                                 '/M.Cortat M.Klay': 'Stephane Klay', 
                                 'S.Klay': 'Stephane Klay',
                                 'S.Klay/R.Cortat': 'Stephane Klay',
                                 'Emest Monin': 'Ernest Monin',
                                 'Belet Monin/': 'Ernest Monin',
                                 '/ Monin': 'Ernest Monin', 
                                 '- Monin': 'Ernest Monin',
                                 'E.Muller': 'Ernest Muller',
                                 'Muller Ernest': 'Ernest Muller',
                                 'E.Muller' : 'Ernest Muller',
                                 'M.Muller' : 'Ernest Muller',
                                 'Muller M.' : 'Ernest Muller',
                                 'EMulter/S.Rufer': "Ernest Muller",
                                 'Muller/ Ernest' : 'Ernest Muller',
                                 'J-Pierre Pataoner' : 'Jean-Pierre Pataoner',
                                 'M.Petermann': 'Jose Petermann', 
                                 'Petermann Josee': 'Jose Petermann', 
                                 'J.Petermann': 'Jose Petermann',
                                 'Petermann J.': 'Jose Petermann',
                                 'Petermann Jose': 'Jose Petermann',
                                 'S.Rufer': "S. Rufer", 
                                 '/ S.Rufer': "S. Rufer",
                                 'S.Rufert': "S. Rufer",
                                 'Rufer S.': "S. Rufer",
                                 'S.Rufer. R.Marchand,': "S. Rufer",
                                 '/S.Rufer G.Meyer': "S. Rufer",
                                 '/S.Rufer B.Stettler': "S. Rufer",
                                 '/B.Stettler S.Rufer': "S. Rufer",
                                 '/ S.Rufer/R.Marchand': "S. Rufer",
                                 'S.Rufer/R.Marchand' : "S. Rufer",
                                 'Baumlin/S.Rufer M.': "S. Rufer",
                                 'Schori Messieurs': "Florent Schori",
                                 'Schori Florent': "Florent Schori",
                                 'F.Schori': "Florent Schori",
                                 'Schori F.': "Florent Schori",
                                'Salomon Yves' : 'Yves Salomon', 
                                 'Stelder Joel': 'Joel Stalder',
                                 'Siatder J.': 'Joel Stalder',
                                 'Stalder Joel': 'Joel Stalder', 
                                 'Stalder J.': 'Joel Stalder', 
                                 'J.Stalder': 'Joel Stalder', 
                                 'M.Stalder': 'Joel Stalder',
                                 'Joel M.': 'Joel Stalder',
                                 'J.Stalcier': 'Joel Stalder',
                                 '/ Wermeille': 'Michel Wermeille',
                                'Wermeille M.': 'Michel Wermeille',
                                 'n/a': None, 
                                 'du om': None,
                                })

In [392]:
for c in df.columns:
    print(df[c].unique())

['Temperature mesuree in-situ' 'Escherichia coli' 'Enterocoques'
 'Germes aerobies' 'Chlore libre in-situ' 'Chlore total in-situ' 'pH'
 'Conductivite (25°C)' 'Turbidite' 'Absorption UV 254' 'DOC' 'Oxydabilite'
 'Alcalinite' 'Durete' 'Potassium' 'Sodium' 'Calcium' 'Magnesium'
 'Ammonium' 'Nitrite' 'Phosphate' 'Chlorure' 'Sulfate' 'Nitrate'
 'Durete totale' 'Temperature' '1,1-Dichloroethylene'
 'Trans-1,2-Dichloroethylene' 'Cis-1,2-Dichloroethylene'
 'Chlorure de methylene' 'Chloroforme' '1,1,1 Trichlorethane'
 'Tetrachlorure de carbone' 'Trichlorethylene' 'Dichlorobromomethane'
 'Dibromochloromethane' '1,1,2 Trichlorethane' 'Perchlorethylene'
 'Bromoforme' 'Pesticides communes' 'Alachlor' 'Atrazine' 'Bromoxynil'
 'Chlorbromuron' 'Chlortoluron' 'Chlorpyriphos' 'Cyanazine'
 'Desethylatrazine' 'Deisopropylatrazine' 'Dinoseb' 'Diuron' 'alpha-HCH'
 'beta-HCH' 'alpha-endosulfan' 'beta-endosulfan' 'Endosulfan  sulfate'
 'Fenpropimorphe' 'Hexazinon' 'Ioxynil' 'Isoproturon' 'Lindane' 'Linuron'
 

In [393]:
def from_df_to_relational_table(df):
    df_Analysis = pd.DataFrame(columns=col_Analysis)
    df_Parameter = pd.DataFrame(columns=col_Parameter)
    df_Echantillon = pd.DataFrame(columns=col_Echantillon)
    df_Location = pd.DataFrame(columns=col_Location)
    df_Sampler = pd.DataFrame(columns=col_Sampler)
    
    df_Parameter = df[["Parametre", "Unite", "Limit"]].drop_duplicates().reset_index(drop=True).reset_index(drop=False)
    df_Parameter = df_Parameter.rename(columns={"index": "ID", "Parametre": "Nom", "Unite": "Unit"})
    df_Parameter["Group"] = "None"
    df_Parameter = df_Parameter[col_Parameter]
    
    df_Location = df[["Address", "WaterType", "Treatment", "Village", "Commune", "Distributeur"]].drop_duplicates().reset_index(drop=True).reset_index(drop=False)
    df_Location = df_Location.rename(columns={"index": "ID"})
    df_Location = df_Location[col_Location]
    
    df_Sampler = df[["Sampler"]].drop_duplicates()
    df_Sampler["FirstName"] = df_Sampler.Sampler.map(lambda x: str(x).split()[0] if x else "None")
    df_Sampler["LastName"] = df_Sampler.Sampler.map(lambda x: str(x).split()[1] if x and len(str(x).split())>1 else "None")
    #df_Sampler = df_Sampler.drop(columns=["Sampler"]) # as needed keep for instance
    df_Sampler = df_Sampler.reset_index(drop=True).reset_index(drop=False).rename(columns={"index": "ID"})
    
    df_Echantillon = df[["Address", "Sampler", "Code", "FloconsNb", "Temperature"]].drop_duplicates().reset_index(drop=True).reset_index(drop=False)
    df_Echantillon = df_Echantillon.rename(columns={"index": "ID"})
    address_dict = {v: k for k, v in df_Location.set_index("ID").Address.to_dict().items()}
    sampler_dict = {v: k for k, v in df_Sampler.set_index("ID").Sampler.to_dict().items()}
    df_Echantillon = df_Echantillon.replace({"Address": address_dict, "Sampler": sampler_dict})
    df_Echantillon = df_Echantillon.rename(columns={"Address": "AddressID", "Sampler": "SamplerID"})
    
    df_Analysis = df[["Parametre", "Code", "Unite", "Limit", "Address", "Sampler", "Value", "Date"]].drop_duplicates().reset_index(drop=True).reset_index(drop=False)
    df_Analysis = df_Analysis.rename(columns={"index": "ID"})
    df_Analysis = df_Analysis.replace({"Address": address_dict, "Sampler": sampler_dict})
    df_Analysis = df_Analysis.rename(columns={"Address": "AddressID", "Sampler": "SamplerID"})
    Echantillon_keys = ["Code", "AddressID", "SamplerID"]
    df_Analysis = df_Analysis.merge(df_Echantillon[["ID"]+Echantillon_keys].rename(columns={"ID": "EchantillonID"}), on=Echantillon_keys, how="inner").drop(columns=Echantillon_keys)
    Parameter_keys = ["Nom", "Unit", "Limit"]
    df_Analysis = df_Analysis.rename(columns={"Parametre": "Nom", "Unite": "Unit"}).merge(df_Parameter[["ID"]+Parameter_keys].rename(columns={"ID": "ParamID"}), on=Parameter_keys, how="inner").drop(columns=Parameter_keys)
    df_Analysis = df_Analysis.reset_index(drop=True)
    
    return df_Analysis.drop(columns=["ID"]), df_Parameter.drop(columns=["ID"]), df_Echantillon.drop(columns=["ID"]), df_Location.drop(columns=["ID"]), df_Sampler.drop(columns=["ID", "Sampler"])

In [394]:
df_Analysis, df_Parameter, df_Echantillon, df_Location, df_Sampler = from_df_to_relational_table(df)

In [395]:
df_Analysis.to_excel("analysis.xlsx")
df_Parameter.to_excel("parameter.xlsx")
df_Echantillon.to_excel("echantillon.xlsx")
df_Location.to_excel("location.xlsx")
df_Sampler.to_excel("sampler.xlsx")

In [396]:
df_Analysis

,Value,Date,EchantillonID,ParamID
0,999.0,2012-04-10,0,0
1,999.0,2012-04-10,1,0
2,14.0,2014-10-06,13,0
3,11.0,2014-10-06,14,0
4,12.0,2014-10-06,15,0
...,...,...,...,...
15312,20.0,2013-07-09,1443,154
15313,20.0,2013-07-09,1444,154
15314,999.0,2013-04-29,1445,155
15315,50.0,2013-04-29,1445,155


In [397]:
df_Parameter

,Nom,Group,Unit,Limit
0,Temperature mesuree in-situ,None,°C,999.0
1,Escherichia coli,None,UFC/100ml,0.0
2,Enterocoques,None,UFC/100ml,0.0
3,Germes aerobies,None,UFC/100ml,300.0
4,Chlore libre in-situ,None,mg/l,0.1
...,...,...,...,...
152,"4,4'-DDD",None,μg/l,0.1
153,"4,4'-DDT",None,μg/l,0.1
154,Methoxychlor,None,μg/l,0.1
155,Alachlore,None,μg/l,0.1


In [398]:
df_Echantillon

,AddressID,SamplerID,Code,FloconsNb,Temperature
0,0,0,1892,1,11.7
1,1,0,1893,1,12.7
2,125,0,518,1,999.0
3,3,0,1020,1,999.0
4,4,0,2960,1,999.0
...,...,...,...,...,...
2220,1817,25,193299,2,12.0
2221,1818,25,182298,2,13.5
2222,1331,42,201560,1,999.0
2223,1819,42,201561,1,999.0


In [399]:
df_Location

,Address,WaterType,Treatment,Village,Commune,Distributeur
0,"Bassecourt, eau de réseau, Espace Industriel 3...",Eau brute,UV,Bassecourt,Haute-Sorne,Conseil communal de Haute-Sorne
1,"Bassecourt, eau de réseau, Rue abbé Monnin, FMB",Eau brute,UV,Bassecourt,Haute-Sorne,Conseil communal de Haute-Sorne
2,"SEHA, Réseau",Eau brute,dioxyde de chlore/UF,Grandfontaine,Grandfontaine,Conseil communal de Grandfontaine
3,"SEHA, Stap. Roche d'Or",Eau brute,dioxyde de chlore/UF,Grandfontaine,Grandfontaine,Conseil communal de Grandfontaine
4,"Grandfontaine, Réseau",Eau traitee,None,Grandfontaine,Grandfontaine,Conseil communal de Grandfontaine
...,...,...,...,...,...,...
1816,Sceut. bout de réseau (eau du SEF],Reseau,None,Bassecourt,Haute-Sorne,Conseil communal de Haute-Sorne
1817,Bassecourt. bout de réseau,Reseau,UV,Bassecourt,Haute-Sorne,Conseil communal de Haute-Sorne
1818,"bout de réseau, Courfaivre",Reseau,UV/javel,None,Haute-Sorne,Conseil communal de Haute-Sorne
1819,Source eau brute,Eau brute,Ozone,Cornol,Cornol,Conseil communal de Cornol


In [400]:
df_Sampler

,FirstName,LastName
0,None,None
1,Jacques,Heyer
2,Yves,Salomon
3,Pierre,Stieger
4,Allemann,J.-P.
...,...,...
99,Petermann,M.
100,D.,Monsieur
101,Queloz,M.
102,S.Rufer/D.Queloz,None
